In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
caltrans_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\V2.0\CALTRANS_V2.0\Caltrans_Vegetation_Management_Treatments_2024.gdb"


In [3]:
list(gpd.list_layers(caltrans_input_gdb_path).name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\core.py:130: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured MultiLineString' is converted to 'MultiLineString'
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))


['Caltrans_Vegetation_Management_RoadsideLandscape_ActivitiesTable_2024',
 'Caltrans_Vegetation_Management_RoadsideLandscape_Treatments_2024',
 'Caltrans_Vegetation_Management_Trees_ActivitiesTable_2024',
 'Caltrans_Vegetation_Management_Tree_Treatments_2024']

In [26]:
validation_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\CNRA_1950_2025.gdb"
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_2020_2025.gdb"
reference_layer_name = 'CNRA_enriched_20250626_point'
output_layer_name = 'CNRA_enriched_20250421_polygon'

In [27]:
output_gdf = gpd.read_file(output_gdb_path, driver='openFileGDB', layer=output_layer_name)
#reference_gdf = gpd.read_file(validation_gdb_path, driver='openFileGDB', layer=reference_layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [28]:
output_gdf.ACTIVITY_DESCRIPTION.unique()

array(['BIOMASS_REMOVAL', 'SINGLE_TREE_SELECTION', 'MOWING', 'HANDLINE',
       'THIN_MAN', 'THIN_MECH', 'PRESCRB_HERBIVORY', 'PRUNING',
       'CHIPPING', 'COMM_THIN', 'HABITAT_REVEG', 'INV_PLANT_REMOVAL',
       'HERBICIDE_APP', 'PILE_BURN', 'MASTICATION', 'PILING', 'TREE_FELL',
       'LOP_AND_SCAT', 'BROADCAST_BURN', 'SLASH_DISPOSAL', 'TREE_PLNTING',
       'SITE_PREP', 'LAND_ACQ', 'DOZER_LINE', 'CHAIN_CRUSH',
       'EROSION_CONTROL', 'PEST_CNTRL', 'DISCING', 'WETLAND_RESTOR',
       'CLEARCUT', 'OAK_WDLND_MGMT', 'ROAD_CLEAR', 'NOT_DEFINED',
       'ECO_HAB_RESTORATION', 'ROAD_OBLITERATION', 'STREAM_CHNL_IMPRV',
       'SALVG_HARVEST', 'AMW_AREA_RESTOR', 'EASEMENT', 'FEE_TITLE',
       'SEEDBED_PREP', 'YARDING', 'VARIABLE_RETEN_HARVEST', 'TBD'],
      dtype=object)

In [21]:
output_gdf[(output_gdf.ACTIVITY_DESCRIPTION == "AMW_AREA_RESTOR") + (output_gdf.ACTIVITY_DESCRIPTION == "OAK_WDLND_MGMT")].ACTIVITY_CAT

19118    MECH_HFR
19153    MECH_HFR
19156    MECH_HFR
19886    MECH_HFR
20012    MECH_HFR
22160    MECH_HFR
22161    MECH_HFR
22163    MECH_HFR
22166    MECH_HFR
23023    MECH_HFR
24483    MECH_HFR
24920    MECH_HFR
25054    MECH_HFR
25055    MECH_HFR
25056    MECH_HFR
25057    MECH_HFR
25385    MECH_HFR
25389    MECH_HFR
26716    MECH_HFR
26731    MECH_HFR
26736    MECH_HFR
26821    MECH_HFR
26822    MECH_HFR
26823    MECH_HFR
26828    MECH_HFR
26829    MECH_HFR
26830    MECH_HFR
26831    MECH_HFR
26832    MECH_HFR
26833    MECH_HFR
Name: ACTIVITY_CAT, dtype: object

In [24]:
reference_gdf.Year_txt.unique()

array(['2024'], dtype=object)

In [21]:
reference_gdf[(reference_gdf['COUNTS_TO_MAS'] == 'YES')]

array(['2024'], dtype=object)

In [23]:
output_gdf[(output_gdf['COUNTS_TO_MAS'] == 'YES') & (output_gdf['Year_txt'] == '2023')]

,PROJECTID_USER,AGENCY,ORG_ADMIN_p,PROJECT_CONTACT,PROJECT_EMAIL,ADMINISTERING_ORG,PROJECT_NAME,PROJECT_STATUS,PROJECT_START,PROJECT_END,...,Source,Year,Year_txt,Act_Code,Crosswalk,Federal_FY,State_FY,TRMT_GEOM,COUNTS_TO_MAS,geometry
221,PFIRS-655,DOD,DOD,Jason Branz,jason.branz@arb.ca.gov,DOD,2023 - MSM Airfield Rx,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Broadcast Burn,2023,2024,POINT,YES,POINT (268520.328 -567364.867)
245,PFIRS-712,DOD,DOD,Jason Branz,jason.branz@arb.ca.gov,DOD,Hotel,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Broadcast Burn,2023,2023,POINT,YES,POINT (251466.775 -508448.569)
330,PFIRS-977,DOD,DOD,Jason Branz,jason.branz@arb.ca.gov,DOD,WWTP,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Hand Pile Burn,2023,2024,POINT,YES,POINT (193878.354 -348117.531)
333,PFIRS-980,DOD,DOD,Jason Branz,jason.branz@arb.ca.gov,DOD,Mojave Creek,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Broadcast Burn,2023,2023,POINT,YES,POINT (190003.819 -342317.914)
334,PFIRS-981,DOD,DOD,Jason Branz,jason.branz@arb.ca.gov,DOD,Branch Pond Rx,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Broadcast Burn,2024,2024,POINT,YES,POINT (190047.626 -352644.13)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10502,PFIRS-18308,None,TIMBER,Jason Branz,jason.branz@arb.ca.gov,TIMBER,2023-Meamber,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Broadcast Burn,2024,2024,POINT,YES,POINT (-248822.637 409643.061)
11545,PFIRS-19608,None,TIMBER,Jason Branz,jason.branz@arb.ca.gov,TIMBER,North Miller Ridge,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Machine Pile Burn,2024,2024,POINT,YES,POINT (-290740.76 78012.952)
11546,PFIRS-19611,None,TIMBER,Jason Branz,jason.branz@arb.ca.gov,TIMBER,North Miller Ridge,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Machine Pile Burn,2024,2024,POINT,YES,POINT (-290740.76 78012.952)
11547,PFIRS-19613,None,TIMBER,Jason Branz,jason.branz@arb.ca.gov,TIMBER,North Miller Ridge,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Machine Pile Burn,2024,2024,POINT,YES,POINT (-290740.76 78012.952)


In [37]:
def val_group(reference_gdb_path, 
              output_gdb_path, 
              reference_layer_name,
              output_layer_name,
              col_sum='ACTIVITY_QUANTITY', 
              group=['ADMINISTERING_ORG', 'BROAD_VEGETATION_TYPE', 'PRIMARY_OBJECTIVE', "YEAR_TXT"]):
    
    reference_gdf = gpd.read_file(reference_gdb_path, driver='openFileGDB', layer=reference_layer_name)
    output_gdf = gpd.read_file(output_gdb_path, driver='openFileGDB', layer=output_layer_name)
    
    reference_gdf.columns = [col.upper() for col in reference_gdf.columns]
    
    # mod pfirs reference
    reference_gdf.loc[reference_gdf['ADMINISTERING_ORG'] == 'US Department of Energy', 'ADMINISTERING_ORG'] = 'DOE'
    output_gdf.columns = [col.upper() for col in output_gdf.columns]
    
    out_group = output_gdf[output_gdf['COUNTS_TO_MAS'] == 'YES'][[col_sum] + group].groupby(group).sum().reset_index()
    ref_group = reference_gdf[reference_gdf['COUNTS_TO_MAS'] == 'YES'][[col_sum] + group].groupby(group).sum().reset_index()
    
    merge_group = out_group.merge(ref_group, on=group, how='outer')
    
    print('out of {} matching combinations of {}, there are {} combinations that have different sum activity quantity'.format(len(merge_group),
                                                                                                                              group,
                                                                                                                             len(merge_group[merge_group['ACTIVITY_QUANTITY_x'] != merge_group['ACTIVITY_QUANTITY_y']]),
                                                                                                                             ))
    
    diff_df = merge_group[merge_group['ACTIVITY_QUANTITY_x'] != merge_group['ACTIVITY_QUANTITY_y']]
    diff_df['DIFF'] = diff_df['ACTIVITY_QUANTITY_x'].fillna(0) - diff_df['ACTIVITY_QUANTITY_y'].fillna(0)
    
    return diff_df

In [38]:
diff_df = val_group(validation_gdb_path,
                   output_gdb_path,
                   reference_layer_name,
                   output_layer_name)

out of 21 matching combinations of ['ADMINISTERING_ORG', 'BROAD_VEGETATION_TYPE', 'PRIMARY_OBJECTIVE', 'YEAR_TXT'], there are 0 combinations that have different sum activity quantity


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [33]:
diff_df[diff_df['YEAR_TXT'] =='2022']

,ADMINISTERING_ORG,BROAD_VEGETATION_TYPE,PRIMARY_OBJECTIVE,YEAR_TXT,ACTIVITY_QUANTITY_x,ACTIVITY_QUANTITY_y,DIFF
121,NPS,FOREST,OTHER_FUELS_REDUCTION,2022,2566.0,NaN,2566.0
147,NPS,GRASS_HERB,OTHER_FUELS_REDUCTION,2022,100.0,NaN,100.0
170,NPS,SHRB_CHAP,OTHER_FUELS_REDUCTION,2022,13.0,NaN,13.0
183,NPS,SPARSE,OTHER_FUELS_REDUCTION,2022,30.0,NaN,30.0
204,NPS,URBAN,OTHER_FUELS_REDUCTION,2022,45.0,NaN,45.0


In [10]:
reference_gdf = gpd.read_file(validation_gdb_path, driver='openFileGDB', layer=reference_layer_name)
output_gdf = gpd.read_file(output_gdb_path, driver='openFileGDB', layer=output_layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [12]:
col_sum='ACTIVITY_QUANTITY'
group=['ADMINISTERING_ORG', 'BROAD_VEGETATION_TYPE', 'PRIMARY_OBJECTIVE', "YEAR_TXT"]

In [3]:
output_gdb_path = r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\USFS_1950_2025.gdb'

In [6]:
gpd.list_layers(output_gdb_path)

,name,geometry_type
0,USFS_Region04_enriched_20250509,MultiPolygon
1,USFS_Region05_enriched_20250509,MultiPolygon
2,USFS_Region06_enriched_20250509,MultiPolygon


In [17]:
output_layer_name = 'USFS_Region05_enriched_20250509'

In [18]:
output_gdf = gpd.read_file(output_gdb_path, driver='openFileGDB', layer=output_layer_name)
output_gdf.columns = [col.upper() for col in output_gdf.columns]

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [19]:
output_gdf[output_gdf['COUNTS_TO_MAS'] == 'YES'][[col_sum] + group].groupby(group).sum().reset_index()

,ADMINISTERING_ORG,BROAD_VEGETATION_TYPE,PRIMARY_OBJECTIVE,YEAR_TXT,ACTIVITY_QUANTITY
0,USFS,AGRICULTURE,OTHER_FOREST_MGMT,1996,23.0
1,USFS,AGRICULTURE,OTHER_FOREST_MGMT,1998,24.0
2,USFS,AGRICULTURE,OTHER_FOREST_MGMT,1999,24.0
3,USFS,AGRICULTURE,OTHER_FOREST_MGMT,2000,32.0
4,USFS,AGRICULTURE,OTHER_FOREST_MGMT,2007,42.0
...,...,...,...,...,...
1429,USFS,WETLAND,REFORESTATION,1996,12.0
1430,USFS,WETLAND,REFORESTATION,1998,5.0
1431,USFS,WETLAND,REFORESTATION,1999,12.0
1432,USFS,WETLAND,REFORESTATION,2001,3.0
